<a href="https://colab.research.google.com/github/juwetta/DLI_Group-B/blob/main/DLI_Malicious_URL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Import imporant libraries

In [2]:
import os
from sklearn.datasets import load_svmlight_file
import glob
import scipy.sparse
import numpy as np

[code below is used for lower the size of dataset, since the full dataset can used up all the RAM just to storing them]

In [3]:
# import shutil

# # Define the source folder path
# source_folder = '/content/drive/My Drive/DLI Group B/url_svmlight'

# # Define the destination folder path
# destination_folder = '/content/drive/My Drive/DLI Group B/url_dataset'

# # Create the destination folder if it doesn't exist
# os.makedirs(destination_folder, exist_ok=True)

# # List of files to copy (Day0.svm to Day60.svm and FeatureTypes)
# files_to_copy = [f'Day{i}.svm' for i in range(61)] + ['FeatureTypes']

# print(f"Copying files to: {destination_folder}")

# # Copy the specified files
# for file_name in files_to_copy:
#     source_file_path = os.path.join(source_folder, file_name)
#     destination_file_path = os.path.join(destination_folder, file_name)
#     try:
#         shutil.copy2(source_file_path, destination_file_path)
#         print(f"Copied: {file_name}")
#     except FileNotFoundError:
#         print(f"Warning: File not found - {file_name}. Skipping.")
#     except Exception as e:
#         print(f"Error copying file {file_name}: {e}")

# print("File copying process completed.")

Accessing to folder

In [4]:
folder_path = '/content/drive/My Drive/DLI Group B/url_dataset'

svm_files = glob.glob(os.path.join(folder_path, "*.svm"))
print(f"Found {len(svm_files)} SVM files in: {folder_path}")

Found 21 SVM files in: /content/drive/My Drive/DLI Group B/url_dataset


In the context of the output, "shape" refers to the dimensions of the data structure (like a NumPy array or a SciPy sparse matrix).

For a 2D structure like combined_X, the shape (2396130, 3231961) means it has 2,396,130 rows and 3,231,961 columns. In this dataset, the rows represent the samples (e.g., URLs), and the columns represent the features.
For a 1D structure like combined_y, the shape (2396130,) means it has 2,396,130 elements. This corresponds to the labels for each of the samples in combined_X.
So, the shape tells you how many samples you have and how many features or labels are associated with each sample.

Read SVM file and store dataset


the different SVM files have varying numbers of features, causing an error when trying to combine them. I'll add a step to determine the total number of features across all files and then load each file with that consistent number of features.

In [5]:
# max_features = 0

# for file_path in svm_files:
#   try:
#     X, _ = load_svmlight_file(file_path)
#     if X.shape[1] > max_features:
#       max_features = X.shape[1]

#   except Exception as e:
#     print(f"Error loading file {os.path.basename(file_path)}: {e}")


# print(f"Maximum number of features found: {max_features}") #74.777s used
max_features = 3231961
print(f"Maximum number of features found: {max_features}")

Maximum number of features found: 3231961


In [6]:

all_X = []
all_y = []
max_features = 3231961
try:
    print("\nLoading and combining data...")
    counter = 0
    # Load and combine data in a single pass, specifying the number of features
    for file_path in svm_files:
        try:
            if counter == 3: break
            X, y = load_svmlight_file(file_path, n_features=max_features)
            all_X.append(X)
            all_y.append(y)
            print(f"{os.path.basename(file_path)}", end="| ")
            counter += 1
        except Exception as e:
            print(f"Error loading file {os.path.basename(file_path)}: {e}")

    if all_X and all_y:
        # Vertically stack the sparse feature matrices
        combined_X = scipy.sparse.vstack(all_X)
        # Concatenate the label arrays
        combined_y = np.concatenate(all_y)

        print("\nSuccessfully combined data from all files.")
        print(f"Shape of combined data (X): {combined_X.shape}")
        print(f"Shape of combined labels (y): {combined_y.shape}")
    else:
        print("\nNo data was loaded from the SVM files.")


except FileNotFoundError:
    print(f"Folder not found at: {folder_path}")
except Exception as e:
    print(f"An error occurred: {e}") #75.54s used


Loading and combining data...
Day0.svm| Day1.svm| Day2.svm| 
Successfully combined data from all files.
Shape of combined data (X): (56000, 3231961)
Shape of combined labels (y): (56000,)


Identify the indexes of real-valued features

In [7]:
import os

feature_types_path = '/content/drive/My Drive/DLI Group B/url_dataset/FeatureTypes'
real_valued_feature_indices = set()

try:
    with open(feature_types_path, 'r') as f:
        for line in f:
            # Assuming each line in FeatureTypes is a feature index
            try:
                index = int(line.strip())
                real_valued_feature_indices.add(index)
            except ValueError:
                # Handle potential non-integer lines in the file
                print(f"Skipping non-integer line in FeatureTypes: {line.strip()}")

    print(f"Identified {len(real_valued_feature_indices)} real-valued feature indices.")
    # print("First 10 real-valued feature indices:", list(real_valued_feature_indices)[:10]) # Optional: print a few indices

except FileNotFoundError:
    print(f"FeatureTypes file not found at: {feature_types_path}")
except Exception as e:
    print(f"An error occurred while reading FeatureTypes: {e}")

# Now you have the set of real-valued feature indices and can use it
# For example, you could filter your data or analyze these specific features.

Identified 64 real-valued feature indices.


Briefly explore the dataset

In [8]:
# Select the first few rows to inspect
num_rows_to_inspect = 5
sample_rows = combined_X[:num_rows_to_inspect]

print(f"Values of real-valued features in the first {num_rows_to_inspect} rows:")

# Iterate through the selected rows
for i in range(sample_rows.shape[0]):
    print(f"\nRow {i+1}:")
    # Iterate through the real-valued feature indices
    for feature_index in sorted(list(real_valued_feature_indices)): # Sorting for consistent output
        # Check if the feature exists in the current row (i.e., it's non-zero)

        if feature_index in sample_rows[i].indices:
            # Get the index within the non-zero elements
            data_index = np.where(sample_rows[i].indices == feature_index)[0][0]
            # Get the value of the feature
            feature_value = sample_rows[i].data[data_index]
            print(f"  Feature {feature_index}: {feature_value}")
        # If the feature index is not in sample_rows[i].indices, its value is 0 in the sparse matrix,
        # so we don't need to explicitly print 0 unless we want to see all real-valued features
        # even if their value is 0 for that sample. Let's only print non-zero real-valued features.

Values of real-valued features in the first 5 rows:

Row 1:
  Feature 4: 0.124138
  Feature 5: 0.117647
  Feature 16: 0.749633
  Feature 17: 0.843029
  Feature 18: 0.197344
  Feature 21: 0.142857
  Feature 22: 0.142857
  Feature 55: 1.0
  Feature 63: 1.0
  Feature 69: 1.0
  Feature 71: 1.0
  Feature 73: 1.0
  Feature 75: 1.0
  Feature 81: 1.0
  Feature 83: 1.0
  Feature 85: 1.0
  Feature 87: 1.0
  Feature 89: 1.0
  Feature 91: 1.0
  Feature 93: 1.0
  Feature 95: 1.0
  Feature 101: 1.0
  Feature 103: 1.0
  Feature 105: 1.0
  Feature 107: 1.0
  Feature 109: 1.0
  Feature 111: 1.0

Row 2:
  Feature 4: 0.103448
  Feature 5: 0.176471
  Feature 16: 0.72266
  Feature 17: 0.836498
  Feature 18: 0.6189
  Feature 21: 0.0119048
  Feature 23: 1.0
  Feature 55: 1.0
  Feature 61: 1.0
  Feature 63: 1.0
  Feature 65: 1.0
  Feature 67: 1.0
  Feature 69: 1.0
  Feature 71: 1.0
  Feature 73: 1.0
  Feature 75: 1.0
  Feature 81: 1.0
  Feature 83: 1.0
  Feature 85: 1.0
  Feature 87: 1.0
  Feature 89: 1.0
  F

# Task
Balance the classes in the `combined_y` variable of the dataset.

## Check class distribution

### Subtask:
Analyze the current distribution of the target variable (`combined_y`) to see how imbalanced the classes are.


**Reasoning**:
Calculate and print the counts of each unique class in the `combined_y` array to understand the class distribution.



In [9]:
unique_classes, class_counts = np.unique(combined_y, return_counts=True)

print("Class distribution in combined_y:")
for class_val, count in zip(unique_classes, class_counts):
    print(f"Class {int(class_val)}: {count}")

Class distribution in combined_y:
Class -1: 35563
Class 1: 20437


## Choose a balancing technique

### Subtask:
Decide on an appropriate method for balancing the classes, such as oversampling (e.g., SMOTE), undersampling, or a combination.


## Apply the balancing technique

### Subtask:
Apply the balancing technique


**Reasoning**:
To balance the dataset, I will first import the necessary libraries, `SMOTE` and `RandomUnderSampler`. Then, I will apply SMOTE to oversample the minority class, followed by RandomUnderSampler to undersample the majority class, as per the instructions. This will create a more balanced dataset for model training.



In [10]:
from imblearn.under_sampling import NearMiss

# Instantiate NearMiss
# NearMiss has different versions (1, 2, 3). You can experiment with them.
# NearMiss-1 selects samples from the majority class whose average distance to the k nearest samples of the minority class is smallest.
# NearMiss-2 selects samples from the majority class whose average distance to the k farthest samples of the minority class is smallest.
# NearMiss-3 selects samples from the majority class whose distance to the k nearest samples of the minority class is largest.
# Let's start with NearMiss-1.
nm = NearMiss(version=1, sampling_strategy='auto', n_neighbors=1) # Adjust n_neighbors if needed

print("Applying NearMiss undersampling...")
# temp_combined_X, temp_combined_y = combined_X, combined_y
# Apply NearMiss to the data
# NearMiss works directly on the original data
X_resampled, y_resampled = nm.fit_resample(temp_combined_X, temp_combined_X)

# Print the shapes of the resampled data to verify
print(f"Shape of X after NearMiss: {X_resampled.shape}")
print(f"Shape of y after NearMiss: {y_resampled.shape}")

# Verify the new class distribution
unique_classes_resampled, class_counts_resampled = np.unique(y_resampled, return_counts=True)
print("\nClass distribution after resampling:")
for class_val, count in zip(unique_classes_resampled, class_counts_resampled):
    print(f"Class {int(class_val)}: {count}")

Applying NearMiss undersampling...
Shape of X after NearMiss: (40874, 3231961)
Shape of y after NearMiss: (40874,)

Class distribution after resampling:
Class -1: 20437
Class 1: 20437


## Train a classification model

Train a classification model on the balanced dataset.

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Split the resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42, stratify=y_resampled)

print("Training Logistic Regression model...")

# Initialize and train the Logistic Regression model
# Due to the large number of features, we might need to adjust solver and max_iter
model = LogisticRegression(solver='liblinear', max_iter=1000, random_state=42)
model.fit(X_train, y_train)

print("Model training complete.")

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
print("\nModel Evaluation:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Training Logistic Regression model...
Model training complete.

Model Evaluation:
Accuracy: 0.9842615999347631

Confusion Matrix:
 [[5980  152]
 [  41 6090]]

Classification Report:
               precision    recall  f1-score   support

        -1.0       0.99      0.98      0.98      6132
         1.0       0.98      0.99      0.98      6131

    accuracy                           0.98     12263
   macro avg       0.98      0.98      0.98     12263
weighted avg       0.98      0.98      0.98     12263



## Model Evaluation

Analyze the performance of the trained model using appropriate metrics for imbalanced datasets.

## Train a Support Vector Machine Model

Train a Linear Support Vector Classification model on the balanced dataset.

In [13]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("Training Linear SVC model...")

# Initialize and train the LinearSVC model
# Adjust max_iter if convergence warnings occur
svm_model = LinearSVC(max_iter=1000, random_state=42)
svm_model.fit(X_train, y_train)

print("Model training complete.")

# Make predictions on the test set
y_pred_svm = svm_model.predict(X_test)

# Evaluate the model
print("\nLinear SVC Model Evaluation:")
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_svm))
print("\nClassification Report:\n", classification_report(y_test, y_pred_svm))

Training Linear SVC model...
Model training complete.

Linear SVC Model Evaluation:
Accuracy: 0.9855663377640056

Confusion Matrix:
 [[5988  144]
 [  33 6098]]

Classification Report:
               precision    recall  f1-score   support

        -1.0       0.99      0.98      0.99      6132
         1.0       0.98      0.99      0.99      6131

    accuracy                           0.99     12263
   macro avg       0.99      0.99      0.99     12263
weighted avg       0.99      0.99      0.99     12263



/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


## Linear SVC Model Evaluation

Analyze the performance of the trained Linear SVC model.